In [1]:
!git clone https://github.com/sakanaowo/PlantXViT

Cloning into 'PlantXViT'...
remote: Enumerating objects: 50365, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 50365 (delta 36), reused 77 (delta 21), pack-reused 50271 (from 1)
Receiving objects: 100% (50365/50365), 1.66 GiB | 15.26 MiB/s, done.
Resolving deltas: 100% (30421/30421), done.
Updating files: 100% (50037/50037), done.


In [33]:
!git pull

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 6 (delta 5), reused 6 (delta 5), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 541 bytes | 541.00 KiB/s, done.
From https://github.com/sakanaowo/PlantXViT
   d3151181..c9909ad8  main       -> origin/main
Updating d3151181..c9909ad8
Fast-forward
 src/model.py                | 2 +-
 utils/preprocess_embrapa.py | 2 ++
 2 files changed, 3 insertions(+), 1 deletion(-)


In [2]:
%cd PlantXViT

/content/PlantXViT


preprocess here

In [34]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.0),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [35]:
!ls ./data/raw/embrapa

test  train  val


In [36]:
import os

In [37]:
root_dir="./data/raw/embrapa"

In [38]:
train_dataset = datasets.ImageFolder(os.path.join(root_dir, "train"), transform=image_transforms)
val_dataset = datasets.ImageFolder(os.path.join(root_dir, "val"), transform=image_transforms)
test_dataset = datasets.ImageFolder(os.path.join(root_dir, "test"), transform=image_transforms)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [39]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import VGG16_Weights


# inception block (chỉnh sửa: tổng output channels = 512)
class InceptionBlock(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 192, kernel_size=1)  # Tăng lên 192

        self.branch3x3 = nn.Sequential(
            nn.Conv2d(in_channels, 160, kernel_size=(1, 3), padding=(0, 1)),
            nn.Conv2d(160, 160, kernel_size=(3, 1), padding=(1, 0)),
        )

        self.branch_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, 160, kernel_size=1),  # Tăng lên 160
        )

    def forward(self, x):
        b1 = self.branch1x1(x)
        b2 = self.branch3x3(x)
        b3 = self.branch_pool(x)
        return torch.cat([b1, b2, b3], dim=1)  # Output shape: (B, 512, H, W)


# patch embedding: split patch -> Linear
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels, patch_size=5, emb_size=16):
        super().__init__()
        self.patch_size = patch_size
        self.emb_size = emb_size
        self.proj = nn.Linear(in_channels * patch_size * patch_size, emb_size)

    def forward(self, x):
        B, C, H, W = x.shape
        x = x.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size)
        x = x.permute(0, 2, 3, 1, 4, 5).contiguous()
        x = x.view(B, -1, C * self.patch_size * self.patch_size)
        return self.proj(x)  # shape: (b,num patches,emb size)


# -------- Transformer Encoder Block (ViT block) --------
class TransformerBlock(nn.Module):
    def __init__(self, emb_size=16, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(emb_size)
        self.attn = nn.MultiheadAttention(emb_size, num_heads=2, batch_first=True)
        self.norm2 = nn.LayerNorm(emb_size)
        self.mlp = nn.Sequential(
            nn.Linear(emb_size, emb_size * 2),
            nn.GELU(),
            nn.Linear(emb_size * 2, emb_size),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x_attn, _ = self.attn(self.norm1(x), self.norm1(x), self.norm1(x))
        x = x + x_attn
        x = x + self.mlp(self.norm2(x))
        return x


# -------- PlantXViT Model --------
class PlantXViT(nn.Module):
    def __init__(self, num_classes=4, patch_size=5, emb_size=16, num_blocks=4, dropout=0.1):
        super().__init__()

        # VGG16 (2 blocks)
        vgg = models.vgg16(weights=VGG16_Weights.DEFAULT)
        self.vgg_block = nn.Sequential(*list(vgg.features[:10]))
        # self.vgg_block = nn.Sequential(*vgg[:10])  # output: (B, 128, 56, 56)

        # Inception-like block → (B, 384, 56, 56)
        self.inception = InceptionBlock(in_channels=128)

        # Patch Embedding → (B, 121, 16)
        # self.patch_embed = PatchEmbedding(in_channels=384, patch_size=patch_size, emb_size=emb_size)
        self.patch_embed = PatchEmbedding(in_channels=512, patch_size=patch_size, emb_size=emb_size)

        # Transformer blocks
        self.transformer = nn.Sequential(*[TransformerBlock(emb_size, dropout) for _ in range(num_blocks)])

        # Classification head
        self.norm = nn.LayerNorm(emb_size)
        self.global_pool = nn.AdaptiveAvgPool1d(1)  # (B, emb_size, 1)
        self.classifier = nn.Linear(emb_size, num_classes)

    def forward(self, x):
        x = self.vgg_block(x)  # (B, 128, 56, 56)
        x = self.inception(x)  # (B, 384, 56, 56)
        x = self.patch_embed(x)  # (B, 121, 16)
        x = self.transformer(x)  # (B, 121, 16)
        x = self.norm(x)  # (B, 121, 16)
        x = x.permute(0, 2, 1)  # (B, 16, 121)
        x = self.global_pool(x).squeeze(-1)  # (B, 16)
        return self.classifier(x)  # (B, num_classes)


In [40]:
model = PlantXViT(num_classes=93)
criterion=nn.CrossEntropyLoss()

training from here

In [10]:
!cat utils/config_loader.py

import yaml


def load_config(path="../configs/config.yaml"):
    with open(path, "r") as f:
        return yaml.safe_load(f)


In [41]:
from utils.config_loader import load_config
config=load_config('./configs/config.yaml')

In [42]:
print(config['output']['embrapa']['model_path'])

./outputs/embrapa/models/plantxvit_best.pth


In [46]:
import os

# ...

MODEL_PATH = "./outputs/embrapa/models/plantxvit_best.pth"

# Tạo thư mục nếu chưa tồn tại
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)


In [44]:
!ls outputs/embrapa/models

plantxvit_best.pth


In [45]:
DATA_DIR=root_dir
BATCH_SIZE=16
EPOCHS=50
LR=1e-4
NUM_CLASSES=93
DEVICE=torch.device('cuda')
MODEL_PATH=config['output']['embrapa']['model_path']

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from tqdm import tqdm

In [48]:

model.to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [49]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss, correct, total = 0, 0, 0

    for inputs, labels in tqdm(loader, desc="Training"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / total
    acc = correct / total
    return avg_loss, acc




In [50]:
def evaluate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0, 0, 0

    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc="Evaluating"):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = running_loss / total
    acc = correct / total
    return avg_loss, acc


In [51]:
best_val_acc = 0
patience,wait=5,0

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")

    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), MODEL_PATH)
        print(f"✅ Saved best model to {MODEL_PATH}")
        wait=0
    else:
      wait+=1
      if wait>=patience:
        print(f"Early stopping at epoch {epoch+1}")
        break



Epoch 1/50


Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.84it/s]


Train Loss: 3.3801 | Acc: 0.3026
Val   Loss: 2.7352 | Acc: 0.4011
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 2/50


Evaluating: 100%|██████████| 466/466 [00:09<00:00, 49.20it/s]


Train Loss: 2.3547 | Acc: 0.4647
Val   Loss: 2.0359 | Acc: 0.5218
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 3/50


...

Train Loss: 0.3394 | Acc: 0.8998
Val   Loss: 0.5301 | Acc: 0.8449

Epoch 29/50


Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.95it/s]


Train Loss: 0.3372 | Acc: 0.8993
Val   Loss: 0.5292 | Acc: 0.8449

Epoch 30/50


Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.56it/s]


Train Loss: 0.3245 | Acc: 0.9036
Val   Loss: 0.5124 | Acc: 0.8487
Early stopping at epoch 30


In [ ]:
import re
log_text="""
Epoch 1/50

Training: 100%|██████████| 1851/1851 [01:15<00:00, 24.49it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.84it/s]

Train Loss: 3.3801 | Acc: 0.3026
Val   Loss: 2.7352 | Acc: 0.4011
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 2/50

Training: 100%|██████████| 1851/1851 [01:17<00:00, 23.93it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 49.20it/s]

Train Loss: 2.3547 | Acc: 0.4647
Val   Loss: 2.0359 | Acc: 0.5218
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 3/50

Training: 100%|██████████| 1851/1851 [01:18<00:00, 23.59it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 49.17it/s]

Train Loss: 1.8078 | Acc: 0.5763
Val   Loss: 1.6736 | Acc: 0.6189
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 4/50

Training: 100%|██████████| 1851/1851 [01:17<00:00, 23.99it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.52it/s]

Train Loss: 1.4754 | Acc: 0.6446
Val   Loss: 1.3858 | Acc: 0.6567
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 5/50

Training: 100%|██████████| 1851/1851 [01:15<00:00, 24.38it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.57it/s]

Train Loss: 1.2590 | Acc: 0.6840
Val   Loss: 1.2172 | Acc: 0.6944
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 6/50

Training: 100%|██████████| 1851/1851 [01:16<00:00, 24.21it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.84it/s]

Train Loss: 1.1021 | Acc: 0.7181
Val   Loss: 1.0689 | Acc: 0.7154
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 7/50

Training: 100%|██████████| 1851/1851 [01:16<00:00, 24.28it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 49.79it/s]

Train Loss: 0.9837 | Acc: 0.7430
Val   Loss: 0.9726 | Acc: 0.7323
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 8/50

Training: 100%|██████████| 1851/1851 [01:15<00:00, 24.42it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.73it/s]

Train Loss: 0.8982 | Acc: 0.7629
Val   Loss: 0.9167 | Acc: 0.7486
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 9/50

Training: 100%|██████████| 1851/1851 [01:17<00:00, 23.78it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 49.00it/s]

Train Loss: 0.8217 | Acc: 0.7797
Val   Loss: 0.8974 | Acc: 0.7493
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 10/50

Training: 100%|██████████| 1851/1851 [01:16<00:00, 24.28it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.78it/s]

Train Loss: 0.7647 | Acc: 0.7932
Val   Loss: 0.7873 | Acc: 0.7827
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 11/50

Training: 100%|██████████| 1851/1851 [01:15<00:00, 24.64it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.71it/s]

Train Loss: 0.7029 | Acc: 0.8075
Val   Loss: 0.7190 | Acc: 0.8034
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 12/50

Training: 100%|██████████| 1851/1851 [01:16<00:00, 24.36it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.29it/s]

Train Loss: 0.6636 | Acc: 0.8162
Val   Loss: 0.7502 | Acc: 0.7901

Epoch 13/50

Training: 100%|██████████| 1851/1851 [01:15<00:00, 24.41it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.16it/s]

Train Loss: 0.6201 | Acc: 0.8264
Val   Loss: 0.7097 | Acc: 0.7995

Epoch 14/50

Training: 100%|██████████| 1851/1851 [01:17<00:00, 23.86it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 49.47it/s]

Train Loss: 0.5852 | Acc: 0.8364
Val   Loss: 0.6622 | Acc: 0.8064
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 15/50

Training: 100%|██████████| 1851/1851 [01:17<00:00, 23.87it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 49.04it/s]

Train Loss: 0.5574 | Acc: 0.8414
Val   Loss: 0.6192 | Acc: 0.8229
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 16/50

Training: 100%|██████████| 1851/1851 [01:17<00:00, 23.96it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.89it/s]

Train Loss: 0.5315 | Acc: 0.8470
Val   Loss: 0.6362 | Acc: 0.8170

Epoch 17/50

Training: 100%|██████████| 1851/1851 [01:18<00:00, 23.48it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 47.78it/s]

Train Loss: 0.5067 | Acc: 0.8536
Val   Loss: 0.5998 | Acc: 0.8266
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 18/50

Training: 100%|██████████| 1851/1851 [01:16<00:00, 24.05it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.14it/s]

Train Loss: 0.4833 | Acc: 0.8605
Val   Loss: 0.6043 | Acc: 0.8231

Epoch 19/50

Training: 100%|██████████| 1851/1851 [01:18<00:00, 23.59it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.20it/s]

Train Loss: 0.4694 | Acc: 0.8654
Val   Loss: 0.5709 | Acc: 0.8341
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 20/50

Training: 100%|██████████| 1851/1851 [01:15<00:00, 24.49it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.12it/s]

Train Loss: 0.4461 | Acc: 0.8701
Val   Loss: 0.5699 | Acc: 0.8346
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 21/50

Training: 100%|██████████| 1851/1851 [01:16<00:00, 24.05it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 49.72it/s]

Train Loss: 0.4284 | Acc: 0.8751
Val   Loss: 0.6039 | Acc: 0.8237

Epoch 22/50

Training: 100%|██████████| 1851/1851 [01:17<00:00, 23.88it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.53it/s]

Train Loss: 0.4131 | Acc: 0.8790
Val   Loss: 0.5662 | Acc: 0.8341

Epoch 23/50

Training: 100%|██████████| 1851/1851 [01:17<00:00, 23.85it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.42it/s]

Train Loss: 0.4023 | Acc: 0.8807
Val   Loss: 0.5573 | Acc: 0.8283

Epoch 24/50

Training: 100%|██████████| 1851/1851 [01:16<00:00, 24.17it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 49.38it/s]

Train Loss: 0.3915 | Acc: 0.8835
Val   Loss: 0.5290 | Acc: 0.8425
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 25/50

Training: 100%|██████████| 1851/1851 [01:16<00:00, 24.35it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 49.24it/s]

Train Loss: 0.3735 | Acc: 0.8900
Val   Loss: 0.5146 | Acc: 0.8510
✅ Saved best model to ./outputs/embrapa/models/plantxvit_best.pth

Epoch 26/50

Training: 100%|██████████| 1851/1851 [01:17<00:00, 23.94it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 49.49it/s]

Train Loss: 0.3659 | Acc: 0.8924
Val   Loss: 0.5108 | Acc: 0.8506

Epoch 27/50

Training: 100%|██████████| 1851/1851 [01:16<00:00, 24.20it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.27it/s]

Train Loss: 0.3527 | Acc: 0.8951
Val   Loss: 0.5477 | Acc: 0.8366

Epoch 28/50

Training: 100%|██████████| 1851/1851 [01:17<00:00, 23.86it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 49.02it/s]

Train Loss: 0.3394 | Acc: 0.8998
Val   Loss: 0.5301 | Acc: 0.8449

Epoch 29/50

Training: 100%|██████████| 1851/1851 [01:17<00:00, 23.83it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.95it/s]

Train Loss: 0.3372 | Acc: 0.8993
Val   Loss: 0.5292 | Acc: 0.8449

Epoch 30/50

Training: 100%|██████████| 1851/1851 [01:17<00:00, 23.84it/s]
Evaluating: 100%|██████████| 466/466 [00:09<00:00, 48.56it/s]

Train Loss: 0.3245 | Acc: 0.9036
Val   Loss: 0.5124 | Acc: 0.8487
Early stopping at epoch 30
"""

# Tìm tất cả các dòng Train và Val
train_lines = re.findall(r'Train Loss: ([\d\.]+) \| Acc: ([\d\.]+)', log_text)
val_lines = re.findall(r'Val\s+Loss: ([\d\.]+) \| Acc: ([\d\.]+)', log_text)

# Tách ra các list số
train_losses = [float(l[0]) for l in train_lines]
train_accuracies = [float(l[1]) for l in train_lines]
val_losses = [float(l[0]) for l in val_lines]
val_accuracies = [float(l[1]) for l in val_lines]

# In kiểm tra
print("Epochs:", len(train_losses))
print("Train Loss (last):", train_losses[-1])
print("Val Accuracy (last):", val_accuracies[-1])